<a href="https://colab.research.google.com/github/sundram2003/SFC_Placement/blob/main/final_sfc_placement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Integer Linear Programming solution**

In [ ]:
!pip install gurobipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 26.6 MB/s eta 0:00:00


In [ ]:
#              +---------+
#              |         |
#              |    R1   |
#              |         |
#              +---------+
#              Capacity: 2
#         Activation Cost: 10
#                 |     |
#                 | Delay: 10 ms
#                 | Forwarding Cost: 1
#                 | Bandwidth: 100
#              +---------+
#              |         |
#              |    R2   |
#              |         |
#              +---------+
#              Capacity: 2
#         Activation Cost: 10
#                 |     |
#                 | Delay: 10 ms
#                 | Forwarding Cost: 1
#                 | Bandwidth: 100
#              +---------+
#              |         |
#              |    R3   |
#              |         |
#              +---------+
#              Capacity: 2
#         Activation Cost: 10

In [ ]:
!pip install gurobipy
from gurobipy import *

import time

start = time.time()

# Define the physical graph
physical_graph = {
    "R1": {"capacity": 2, "activation_cost": 10},
    "R2": {"capacity": 2, "activation_cost": 10},
    "R3": {"capacity": 2, "activation_cost": 10}
}

physical_graph_edges = [
    ("R1", "R2", {"delay": 10, "forwarding_cost": 1, "bandwidth": 100}),
    ("R2", "R3", {"delay": 10, "forwarding_cost": 1, "bandwidth": 100})
]

# Define service requests
service_requests = {
    "SFC1": {"vnfs": ["VNF1", "VNF2"], "dst": "R3", "bandwidth": 50, "delay_constraint": 30},
    "SFC2": {"vnfs": ["VNF3"], "dst": "R2", "bandwidth": 30, "delay_constraint": 20}
}

def solve_ILP(physical_graph, service_requests):
    # Create Gurobi model
    model = Model()

    # Decision variables
    placement = model.addVars([(node, sfc, vnf) for node in physical_graph for sfc in service_requests for vnf in service_requests[sfc]["vnfs"]], vtype=GRB.BINARY, name="placement")
    flow = model.addVars([(src, dst, sfc) for src, dst in [(src, dst) for src, dst, _ in physical_graph_edges] for sfc in service_requests], vtype=GRB.INTEGER, name="flow")

    # Constraints
    # Capacity constraints
    for node in physical_graph:
        model.addConstr(
            sum(placement[node, sfc, vnf] for sfc in service_requests for vnf in service_requests[sfc]["vnfs"]) <= physical_graph[node]["capacity"]
        )

    # Bandwidth constraints
    for src, dst, edge in physical_graph_edges:
        model.addConstr(
            sum(flow[src, dst, sfc] for sfc in service_requests) <= edge["bandwidth"]
        )

    # Delay constraints
    for sfc in service_requests:
        total_delay = sum(flow[src, dst, sfc] * edge["delay"] for src, dst, edge in physical_graph_edges)
        model.addConstr(total_delay <= service_requests[sfc]["delay_constraint"])

    # Flow conservation constraints
    for sfc in service_requests:
        for node in physical_graph:
            incoming_flow = sum(flow[src, node, sfc] for src in physical_graph if (src, node) in [(src, dst) for src, dst, _ in physical_graph_edges])
            outgoing_flow = sum(flow[node, dst, sfc] for dst in physical_graph if (node, dst) in [(src, dst) for src, dst, _ in physical_graph_edges])
            model.addConstr(incoming_flow == outgoing_flow)

        # Destination constraint
        dst_node = service_requests[sfc]["dst"]
        model.addConstr(
            sum(flow[node, dst_node, sfc] for node in physical_graph if (node, dst_node) in [(src, dst) for src, dst, _ in physical_graph_edges]) == sum(placement[dst_node, sfc, vnf] for vnf in service_requests[sfc]["vnfs"])
        )

    # VNF ordering constraints
    for sfc in service_requests:
        vnfs = service_requests[sfc]["vnfs"]
        for i in range(len(vnfs) - 1):
            for node in physical_graph:
                model.addConstr(
                    placement[node, sfc, vnfs[i]] >= placement[node, sfc, vnfs[i + 1]]
                )

    # Objective function
    activation_cost = sum(physical_graph[node]["activation_cost"] * any(placement[node, sfc, vnf] for sfc in service_requests for vnf in service_requests[sfc]["vnfs"]) for node in physical_graph)
    deployment_cost = sum(sum(placement[node, sfc, vnf] for node in physical_graph) for sfc in service_requests for vnf in service_requests[sfc]["vnfs"])
    forwarding_cost = sum(edge["forwarding_cost"] * flow[src, dst, sfc] for src, dst, edge in physical_graph_edges for sfc in service_requests)
    sla_violation_cost = sum(service_requests[sfc]["bandwidth"] * (1 - sum(flow[src, dst, sfc] for src, dst in [(src, dst) for src, dst, _ in physical_graph_edges])) for sfc in service_requests)

    model.setObjective(activation_cost + deployment_cost + forwarding_cost + sla_violation_cost, GRB.MINIMIZE)

    # Solve ILP
    model.optimize()

    # Extract placement and flow variables from solution
    placement_sol = {(node, sfc, vnf): placement[node, sfc, vnf].X for node in physical_graph for sfc in service_requests for vnf in service_requests[sfc]["vnfs"]}
    flow_sol = {(src, dst, sfc): flow[src, dst, sfc].X for src, dst in [(src, dst) for src, dst, _ in physical_graph_edges] for sfc in service_requests}

    return placement_sol, flow_sol

def calculate_cost(physical_graph, service_requests, placement, flow):
    activation_cost = sum(physical_graph[node]["activation_cost"] * any(placement[node, sfc, vnf] for sfc in service_requests for vnf in service_requests[sfc]["vnfs"]) for node in physical_graph)
    deployment_cost = sum(sum(placement[node, sfc, vnf] for node in physical_graph) for sfc in service_requests for vnf in service_requests[sfc]["vnfs"])
    forwarding_cost = sum(edge["forwarding_cost"] * flow[src, dst, sfc] for src, dst, edge in physical_graph_edges for sfc in service_requests)
    sla_violation_cost = sum(service_requests[sfc]["bandwidth"] * (1 - sum(flow[src, dst, sfc] for src, dst in [(src, dst) for src, dst, _ in physical_graph_edges])) for sfc in service_requests)

    total_cost = activation_cost + deployment_cost + forwarding_cost + sla_violation_cost
    return total_cost

def main():
    placement, flow = solve_ILP(physical_graph, service_requests)
    print("Optimal Placement:", placement)
    print("Flow:", flow)

    cost = calculate_cost(physical_graph, service_requests, placement, flow)
    print("Overall Cost:", cost)

if __name__ == "__main__":
    main()

end = time.time()
print("Execution time for ILP :  ",end - start)

Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 18 rows, 13 columns and 36 nonzeros
Model fingerprint: 0xceaeb0db
Variable types: 0 continuous, 13 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 1e+02]
Found heuristic solution: objective 110.0000000
Presolve removed 18 rows and 13 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.09 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 110 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.100000000000e+02, best bound 1.10000000

**Using AC3**

In [ ]:
from typing import Dict, List, Tuple
import time

start = time.time()
# Define the physical graph
physical_graph = {
    "R1": {"capacity": 2, "activation_cost": 10},
    "R2": {"capacity": 2, "activation_cost": 10},
    "R3": {"capacity": 2, "activation_cost": 10}
}

physical_graph_edges = [
    ("R1", "R2", {"delay": 10, "forwarding_cost": 1, "bandwidth": 100}),
    ("R2", "R3", {"delay": 10, "forwarding_cost": 1, "bandwidth": 100})
]

# Define service requests
service_requests = {
    "SFC1": {"vnfs": ["VNF1", "VNF2"], "dst": "R3", "bandwidth": 50, "delay_constraint": 30},
    "SFC2": {"vnfs": ["VNF3"], "dst": "R2", "bandwidth": 30, "delay_constraint": 20}
}

def revise(physical_graph, service_requests, placement, flow):
    revised = False

    # Check capacity constraints
    for node in physical_graph:
        if sum(placement[node, sfc, vnf] for sfc in service_requests for vnf in service_requests[sfc]["vnfs"]) > physical_graph[node]["capacity"]:
            for sfc in service_requests:
                for vnf in service_requests[sfc]["vnfs"]:
                    if placement[node, sfc, vnf] == 1:
                        placement[node, sfc, vnf] = 0
                        revised = True

    # Check bandwidth constraints
    for src, dst, edge in physical_graph_edges:
        used_bandwidth = sum(flow[src, dst, sfc] for sfc in service_requests)
        if used_bandwidth > edge["bandwidth"]:
            for sfc in service_requests:
                if flow[src, dst, sfc] > 0:
                    flow[src, dst, sfc] = 0
                    revised = True

    # Check delay constraints
    for sfc in service_requests:
        total_delay = sum(flow[src, dst, sfc] * edge["delay"] for src, dst, edge in physical_graph_edges)
        if total_delay > service_requests[sfc]["delay_constraint"]:
            for src, dst, _ in physical_graph_edges:
                if flow[src, dst, sfc] > 0:
                    flow[src, dst, sfc] = 0
                    revised = True

    # Check flow conservation constraints
    for sfc in service_requests:
        for node in physical_graph:
            incoming_flow = sum(flow[src, node, sfc] for src in physical_graph if (src, node) in [(src, dst) for src, dst, _ in physical_graph_edges])
            outgoing_flow = sum(flow[node, dst, sfc] for dst in physical_graph if (node, dst) in [(src, dst) for src, dst, _ in physical_graph_edges])
            if incoming_flow != outgoing_flow:
                for src in physical_graph:
                    if (src, node) in [(src, dst) for src, dst, _ in physical_graph_edges]:
                        flow[src, node, sfc] = 0
                for dst in physical_graph:
                    if (node, dst) in [(src, dst) for src, dst, _ in physical_graph_edges]:
                        flow[node, dst, sfc] = 0
                revised = True

    return revised

def ac3(physical_graph, service_requests):
    # Initialize placement and flow variables
    placement = {(node, sfc, vnf): 0 for node in physical_graph for sfc in service_requests for vnf in service_requests[sfc]["vnfs"]}
    flow = {(src, dst, sfc): 0 for src, dst in [(src, dst) for src, dst, _ in physical_graph_edges] for sfc in service_requests}

    queue = [(node, sfc, vnf) for node in physical_graph for sfc in service_requests for vnf in service_requests[sfc]["vnfs"]]
    queue.extend([(src, dst, sfc) for src, dst in [(src, dst) for src, dst, _ in physical_graph_edges] for sfc in service_requests])

    while queue:
        variable = queue.pop(0)
        if revise(physical_graph, service_requests, placement, flow):
            if isinstance(variable, tuple) and len(variable) == 3:
                queue.extend([(node, sfc, vnf) for node in physical_graph for sfc in service_requests for vnf in service_requests[sfc]["vnfs"] if (node, sfc, vnf) != variable])
            else:
                queue.extend([(src, dst, sfc) for src, dst in [(src, dst) for src, dst, _ in physical_graph_edges] for sfc in service_requests if (src, dst, sfc) != variable])

    return placement, flow

def calculate_cost(physical_graph, service_requests, placement, flow):
    activation_cost = sum(physical_graph[node]["activation_cost"] * any(placement[node, sfc, vnf] for sfc in service_requests for vnf in service_requests[sfc]["vnfs"]) for node in physical_graph)
    deployment_cost = sum(sum(placement[node, sfc, vnf] for node in physical_graph) for sfc in service_requests for vnf in service_requests[sfc]["vnfs"])
    forwarding_cost = sum(edge["forwarding_cost"] * flow[src, dst, sfc] for src, dst, edge in physical_graph_edges for sfc in service_requests)
    sla_violation_cost = sum(service_requests[sfc]["bandwidth"] * (1 - sum(flow[src, dst, sfc] for src, dst in [(src, dst) for src, dst, _ in physical_graph_edges])) for sfc in service_requests)

    total_cost = activation_cost + deployment_cost + forwarding_cost + sla_violation_cost
    return total_cost

def main():
    placement, flow = ac3(physical_graph, service_requests)
    print("Placement:", placement)
    print("Flow:", flow)

    cost = calculate_cost(physical_graph, service_requests, placement, flow)
    print("Overall Cost:", cost)

if __name__ == "__main__":
    main()


end = time.time()
print("Execution time for AC3 :  ",end - start)

Placement: {('R1', 'SFC1', 'VNF1'): 0, ('R1', 'SFC1', 'VNF2'): 0, ('R1', 'SFC2', 'VNF3'): 0, ('R2', 'SFC1', 'VNF1'): 0, ('R2', 'SFC1', 'VNF2'): 0, ('R2', 'SFC2', 'VNF3'): 0, ('R3', 'SFC1', 'VNF1'): 0, ('R3', 'SFC1', 'VNF2'): 0, ('R3', 'SFC2', 'VNF3'): 0}
Flow: {('R1', 'R2', 'SFC1'): 0, ('R1', 'R2', 'SFC2'): 0, ('R2', 'R3', 'SFC1'): 0, ('R2', 'R3', 'SFC2'): 0}
Overall Cost: 80
Execution time for AC3 :   0.012274742126464844


In [ ]:
#             +-----+
#             |     |
#             | R1  | Cap: 3
#             |     | Act. Cost: 15
#             +-----+
#                |
#                |
#         +------+------+
#         |              |
# +-------+-------+  +---+---+
# |      R2       |  |       |  Cap: 3
# |Act. Cost: 20  |  |  R3   |  Act. Cost: 15
# | Cap: 4        |  |       |
# +-------+-------+  +---+---+
#         |              |
#         |              |  Delay: 6
#         +------+------+   Fwd. Cost: 2
#                |          Bw: 120
#                |
#             +----------------+
#             |  R4            |
#             |Act. Cost: 2    |
#             |cap 2|          |
#             +-----------------+
#                |
#                |
#         +------+------+
#         |              |
# +-------+-------+  +---+---+
# |                |  |       |
# |      R5       |  |  R6   |
# |                |  |       |
# +----------------+  +-------+
# similarly add all the cost, delays and bandwidths


In [ ]:
physical_graph = {
    "R1": {"capacity": 3, "activation_cost": 15},
    "R2": {"capacity": 4, "activation_cost": 20},
    "R3": {"capacity": 3, "activation_cost": 15},
    "R4": {"capacity": 2, "activation_cost": 10},
    "R5": {"capacity": 3, "activation_cost": 15},
    "R6": {"capacity": 2, "activation_cost": 10}
}

physical_graph_edges = [
    ("R1", "R2", {"delay": 5, "forwarding_cost": 2, "bandwidth": 150}),
    ("R1", "R3", {"delay": 8, "forwarding_cost": 3, "bandwidth": 100}),
    ("R2", "R3", {"delay": 6, "forwarding_cost": 2, "bandwidth": 120}),
    ("R2", "R4", {"delay": 7, "forwarding_cost": 3, "bandwidth": 110}),
    ("R3", "R4", {"delay": 4, "forwarding_cost": 1, "bandwidth": 140}),
    ("R4", "R5", {"delay": 6, "forwarding_cost": 2, "bandwidth": 130}),
    ("R4", "R6", {"delay": 9, "forwarding_cost": 4, "bandwidth": 90}),
    ("R5", "R6", {"delay": 5, "forwarding_cost": 2, "bandwidth": 120})
]
service_requests = {
    "SFC1": {"src": "R1", "vnfs": ["VNF1", "VNF2", "VNF3"], "dst": "R6", "bandwidth": 60, "delay_constraint": 25},
    "SFC2": {"src": "R2", "vnfs": ["VNF4", "VNF5"], "dst": "R5", "bandwidth": 40, "delay_constraint": 20},
    "SFC3": {"src": "R3", "vnfs": ["VNF6"], "dst": "R4", "bandwidth": 30, "delay_constraint": 15}
}

**AC3 for 2nd topology**

In [ ]:
from typing import Dict, List, Tuple
import time

start = time.time()
# Define the physical graph
physical_graph = {
    "R1": {"capacity": 3, "activation_cost": 15},
    "R2": {"capacity": 4, "activation_cost": 20},
    "R3": {"capacity": 3, "activation_cost": 15},
    "R4": {"capacity": 2, "activation_cost": 10},
    "R5": {"capacity": 3, "activation_cost": 15},
    "R6": {"capacity": 2, "activation_cost": 10}
}

physical_graph_edges = [
    ("R1", "R2", {"delay": 5, "forwarding_cost": 2, "bandwidth": 150}),
    ("R1", "R3", {"delay": 8, "forwarding_cost": 3, "bandwidth": 100}),
    ("R2", "R3", {"delay": 6, "forwarding_cost": 2, "bandwidth": 120}),
    ("R2", "R4", {"delay": 7, "forwarding_cost": 3, "bandwidth": 110}),
    ("R3", "R4", {"delay": 4, "forwarding_cost": 1, "bandwidth": 140}),
    ("R4", "R5", {"delay": 6, "forwarding_cost": 2, "bandwidth": 130}),
    ("R4", "R6", {"delay": 9, "forwarding_cost": 4, "bandwidth": 90}),
    ("R5", "R6", {"delay": 5, "forwarding_cost": 2, "bandwidth": 120})
]
service_requests = {
    "SFC1": {"src": "R1", "vnfs": ["VNF1", "VNF2", "VNF3"], "dst": "R6", "bandwidth": 60, "delay_constraint": 25},
    "SFC2": {"src": "R2", "vnfs": ["VNF4", "VNF5"], "dst": "R5", "bandwidth": 40, "delay_constraint": 20},
    "SFC3": {"src": "R3", "vnfs": ["VNF6"], "dst": "R4", "bandwidth": 30, "delay_constraint": 15}
}

def revise(physical_graph, service_requests, placement, flow):
    revised = False

    # Check capacity constraints
    for node in physical_graph:
        if sum(placement[node, sfc, vnf] for sfc in service_requests for vnf in service_requests[sfc]["vnfs"]) > physical_graph[node]["capacity"]:
            for sfc in service_requests:
                for vnf in service_requests[sfc]["vnfs"]:
                    if placement[node, sfc, vnf] == 1:
                        placement[node, sfc, vnf] = 0
                        revised = True

    # Check bandwidth constraints
    for src, dst, edge in physical_graph_edges:
        used_bandwidth = sum(flow[src, dst, sfc] for sfc in service_requests)
        if used_bandwidth > edge["bandwidth"]:
            for sfc in service_requests:
                if flow[src, dst, sfc] > 0:
                    flow[src, dst, sfc] = 0
                    revised = True

    # Check delay constraints
    for sfc in service_requests:
        total_delay = sum(flow[src, dst, sfc] * edge["delay"] for src, dst, edge in physical_graph_edges)
        if total_delay > service_requests[sfc]["delay_constraint"]:
            for src, dst, _ in physical_graph_edges:
                if flow[src, dst, sfc] > 0:
                    flow[src, dst, sfc] = 0
                    revised = True

    # Check flow conservation constraints
    for sfc in service_requests:
        for node in physical_graph:
            incoming_flow = sum(flow[src, node, sfc] for src in physical_graph if (src, node) in [(src, dst) for src, dst, _ in physical_graph_edges])
            outgoing_flow = sum(flow[node, dst, sfc] for dst in physical_graph if (node, dst) in [(src, dst) for src, dst, _ in physical_graph_edges])
            if incoming_flow != outgoing_flow:
                for src in physical_graph:
                    if (src, node) in [(src, dst) for src, dst, _ in physical_graph_edges]:
                        flow[src, node, sfc] = 0
                for dst in physical_graph:
                    if (node, dst) in [(src, dst) for src, dst, _ in physical_graph_edges]:
                        flow[node, dst, sfc] = 0
                revised = True

    return revised

def ac3(physical_graph, service_requests):
    # Initialize placement and flow variables
    placement = {(node, sfc, vnf): 0 for node in physical_graph for sfc in service_requests for vnf in service_requests[sfc]["vnfs"]}
    flow = {(src, dst, sfc): 0 for src, dst in [(src, dst) for src, dst, _ in physical_graph_edges] for sfc in service_requests}

    queue = [(node, sfc, vnf) for node in physical_graph for sfc in service_requests for vnf in service_requests[sfc]["vnfs"]]
    queue.extend([(src, dst, sfc) for src, dst in [(src, dst) for src, dst, _ in physical_graph_edges] for sfc in service_requests])

    while queue:
        variable = queue.pop(0)
        if revise(physical_graph, service_requests, placement, flow):
            if isinstance(variable, tuple) and len(variable) == 3:
                queue.extend([(node, sfc, vnf) for node in physical_graph for sfc in service_requests for vnf in service_requests[sfc]["vnfs"] if (node, sfc, vnf) != variable])
            else:
                queue.extend([(src, dst, sfc) for src, dst in [(src, dst) for src, dst, _ in physical_graph_edges] for sfc in service_requests if (src, dst, sfc) != variable])

    return placement, flow

def calculate_cost(physical_graph, service_requests, placement, flow):
    activation_cost = sum(physical_graph[node]["activation_cost"] * any(placement[node, sfc, vnf] for sfc in service_requests for vnf in service_requests[sfc]["vnfs"]) for node in physical_graph)
    deployment_cost = sum(sum(placement[node, sfc, vnf] for node in physical_graph) for sfc in service_requests for vnf in service_requests[sfc]["vnfs"])
    forwarding_cost = sum(edge["forwarding_cost"] * flow[src, dst, sfc] for src, dst, edge in physical_graph_edges for sfc in service_requests)
    sla_violation_cost = sum(service_requests[sfc]["bandwidth"] * (1 - sum(flow[src, dst, sfc] for src, dst in [(src, dst) for src, dst, _ in physical_graph_edges])) for sfc in service_requests)

    total_cost = activation_cost + deployment_cost + forwarding_cost + sla_violation_cost
    return total_cost

def main():
    placement, flow = ac3(physical_graph, service_requests)
    print("Placement:", placement)
    print("Flow:", flow)

    cost = calculate_cost(physical_graph, service_requests, placement, flow)
    print("Overall Cost:", cost)

if __name__ == "__main__":
    main()


end = time.time()
print("Execution time for AC3 :  ",end - start)

Placement: {('R1', 'SFC1', 'VNF1'): 0, ('R1', 'SFC1', 'VNF2'): 0, ('R1', 'SFC1', 'VNF3'): 0, ('R1', 'SFC2', 'VNF4'): 0, ('R1', 'SFC2', 'VNF5'): 0, ('R1', 'SFC3', 'VNF6'): 0, ('R2', 'SFC1', 'VNF1'): 0, ('R2', 'SFC1', 'VNF2'): 0, ('R2', 'SFC1', 'VNF3'): 0, ('R2', 'SFC2', 'VNF4'): 0, ('R2', 'SFC2', 'VNF5'): 0, ('R2', 'SFC3', 'VNF6'): 0, ('R3', 'SFC1', 'VNF1'): 0, ('R3', 'SFC1', 'VNF2'): 0, ('R3', 'SFC1', 'VNF3'): 0, ('R3', 'SFC2', 'VNF4'): 0, ('R3', 'SFC2', 'VNF5'): 0, ('R3', 'SFC3', 'VNF6'): 0, ('R4', 'SFC1', 'VNF1'): 0, ('R4', 'SFC1', 'VNF2'): 0, ('R4', 'SFC1', 'VNF3'): 0, ('R4', 'SFC2', 'VNF4'): 0, ('R4', 'SFC2', 'VNF5'): 0, ('R4', 'SFC3', 'VNF6'): 0, ('R5', 'SFC1', 'VNF1'): 0, ('R5', 'SFC1', 'VNF2'): 0, ('R5', 'SFC1', 'VNF3'): 0, ('R5', 'SFC2', 'VNF4'): 0, ('R5', 'SFC2', 'VNF5'): 0, ('R5', 'SFC3', 'VNF6'): 0, ('R6', 'SFC1', 'VNF1'): 0, ('R6', 'SFC1', 'VNF2'): 0, ('R6', 'SFC1', 'VNF3'): 0, ('R6', 'SFC2', 'VNF4'): 0, ('R6', 'SFC2', 'VNF5'): 0, ('R6', 'SFC3', 'VNF6'): 0}
Flow: {('R1', 'R

In [ ]:
from gurobipy import *

import time

start = time.time()

# Define the physical graph
physical_graph = {
    "R1": {"capacity": 3, "activation_cost": 15},
    "R2": {"capacity": 4, "activation_cost": 20},
    "R3": {"capacity": 3, "activation_cost": 15},
    "R4": {"capacity": 2, "activation_cost": 10},
    "R5": {"capacity": 3, "activation_cost": 15},
    "R6": {"capacity": 2, "activation_cost": 10}
}

physical_graph_edges = [
    ("R1", "R2", {"delay": 5, "forwarding_cost": 2, "bandwidth": 150}),
    ("R1", "R3", {"delay": 8, "forwarding_cost": 3, "bandwidth": 100}),
    ("R2", "R3", {"delay": 6, "forwarding_cost": 2, "bandwidth": 120}),
    ("R2", "R4", {"delay": 7, "forwarding_cost": 3, "bandwidth": 110}),
    ("R3", "R4", {"delay": 4, "forwarding_cost": 1, "bandwidth": 140}),
    ("R4", "R5", {"delay": 6, "forwarding_cost": 2, "bandwidth": 130}),
    ("R4", "R6", {"delay": 9, "forwarding_cost": 4, "bandwidth": 90}),
    ("R5", "R6", {"delay": 5, "forwarding_cost": 2, "bandwidth": 120})
]
service_requests = {
    "SFC1": {"src": "R1", "vnfs": ["VNF1", "VNF2", "VNF3"], "dst": "R6", "bandwidth": 60, "delay_constraint": 25},
    "SFC2": {"src": "R2", "vnfs": ["VNF4", "VNF5"], "dst": "R5", "bandwidth": 40, "delay_constraint": 20},
    "SFC3": {"src": "R3", "vnfs": ["VNF6"], "dst": "R4", "bandwidth": 30, "delay_constraint": 15}
}

def solve_ILP(physical_graph, service_requests):
    # Create Gurobi model
    model = Model()

    # Decision variables
    placement = model.addVars([(node, sfc, vnf) for node in physical_graph for sfc in service_requests for vnf in service_requests[sfc]["vnfs"]], vtype=GRB.BINARY, name="placement")
    flow = model.addVars([(src, dst, sfc) for src, dst in [(src, dst) for src, dst, _ in physical_graph_edges] for sfc in service_requests], vtype=GRB.INTEGER, name="flow")

    # Constraints
    # Capacity constraints
    for node in physical_graph:
        model.addConstr(
            sum(placement[node, sfc, vnf] for sfc in service_requests for vnf in service_requests[sfc]["vnfs"]) <= physical_graph[node]["capacity"]
        )

    # Bandwidth constraints
    for src, dst, edge in physical_graph_edges:
        model.addConstr(
            sum(flow[src, dst, sfc] for sfc in service_requests) <= edge["bandwidth"]
        )

    # Delay constraints
    for sfc in service_requests:
        total_delay = sum(flow[src, dst, sfc] * edge["delay"] for src, dst, edge in physical_graph_edges)
        model.addConstr(total_delay <= service_requests[sfc]["delay_constraint"])

    # Flow conservation constraints
    for sfc in service_requests:
        for node in physical_graph:
            incoming_flow = sum(flow[src, node, sfc] for src in physical_graph if (src, node) in [(src, dst) for src, dst, _ in physical_graph_edges])
            outgoing_flow = sum(flow[node, dst, sfc] for dst in physical_graph if (node, dst) in [(src, dst) for src, dst, _ in physical_graph_edges])
            model.addConstr(incoming_flow == outgoing_flow)

        # Destination constraint
        dst_node = service_requests[sfc]["dst"]
        model.addConstr(
            sum(flow[node, dst_node, sfc] for node in physical_graph if (node, dst_node) in [(src, dst) for src, dst, _ in physical_graph_edges]) == sum(placement[dst_node, sfc, vnf] for vnf in service_requests[sfc]["vnfs"])
        )

    # VNF ordering constraints
    for sfc in service_requests:
        vnfs = service_requests[sfc]["vnfs"]
        for i in range(len(vnfs) - 1):
            for node in physical_graph:
                model.addConstr(
                    placement[node, sfc, vnfs[i]] >= placement[node, sfc, vnfs[i + 1]]
                )

    # Objective function
    activation_cost = sum(physical_graph[node]["activation_cost"] * any(placement[node, sfc, vnf] for sfc in service_requests for vnf in service_requests[sfc]["vnfs"]) for node in physical_graph)
    deployment_cost = sum(sum(placement[node, sfc, vnf] for node in physical_graph) for sfc in service_requests for vnf in service_requests[sfc]["vnfs"])
    forwarding_cost = sum(edge["forwarding_cost"] * flow[src, dst, sfc] for src, dst, edge in physical_graph_edges for sfc in service_requests)
    sla_violation_cost = sum(service_requests[sfc]["bandwidth"] * (1 - sum(flow[src, dst, sfc] for src, dst in [(src, dst) for src, dst, _ in physical_graph_edges])) for sfc in service_requests)

    model.setObjective(activation_cost + deployment_cost + forwarding_cost + sla_violation_cost, GRB.MINIMIZE)

    # Solve ILP
    model.optimize()

    # Extract placement and flow variables from solution
    placement_sol = {(node, sfc, vnf): placement[node, sfc, vnf].X for node in physical_graph for sfc in service_requests for vnf in service_requests[sfc]["vnfs"]}
    flow_sol = {(src, dst, sfc): flow[src, dst, sfc].X for src, dst in [(src, dst) for src, dst, _ in physical_graph_edges] for sfc in service_requests}

    return placement_sol, flow_sol

def calculate_cost(physical_graph, service_requests, placement, flow):
    activation_cost = sum(physical_graph[node]["activation_cost"] * any(placement[node, sfc, vnf] for sfc in service_requests for vnf in service_requests[sfc]["vnfs"]) for node in physical_graph)
    deployment_cost = sum(sum(placement[node, sfc, vnf] for node in physical_graph) for sfc in service_requests for vnf in service_requests[sfc]["vnfs"])
    forwarding_cost = sum(edge["forwarding_cost"] * flow[src, dst, sfc] for src, dst, edge in physical_graph_edges for sfc in service_requests)
    sla_violation_cost = sum(service_requests[sfc]["bandwidth"] * (1 - sum(flow[src, dst, sfc] for src, dst in [(src, dst) for src, dst, _ in physical_graph_edges])) for sfc in service_requests)

    total_cost = activation_cost + deployment_cost + forwarding_cost + sla_violation_cost
    return total_cost

def main():
    placement, flow = solve_ILP(physical_graph, service_requests)
    print("Optimal Placement:", placement)
    print("Flow:", flow)

    cost = calculate_cost(physical_graph, service_requests, placement, flow)
    print("Overall Cost:", cost)

if __name__ == "__main__":
    main()

end = time.time()
print("Execution time for ILP :  ",end - start)

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 56 rows, 60 columns and 179 nonzeros
Model fingerprint: 0x140e4fce
Variable types: 0 continuous, 60 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e+00, 6e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+02]
Found heuristic solution: objective 215.0000000
Presolve removed 56 rows and 60 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 215 

Optimal solution found (tolerance 1.00e-04)
Best objective 2.150000000000e+02, best bound 2.150000000000e+02, gap 0.0000%
Optimal Placement: {('R1', 'SFC1', 'VNF1'): 0.

In [ ]:
#Final AC-3 Taking input from file
from typing import Dict, List, Tuple
import time

start = time.time()
# Define the physical graph
def read_input_data(input_file):
    with open(input_file, 'r') as file:
        file_content = file.read()
        exec(file_content, globals())
    return nodes_data, edges_data, requests_data

def generate_physical_graph(nodes_data):
    physical_graph = {}
    for node, data in nodes_data.items():
        physical_graph[node] = {"capacity": data["capacity"], "activation_cost": data["activation_cost"]}
    return physical_graph

def generate_physical_graph_edges(edges_data):
    physical_graph_edges = []
    for edge in edges_data:
        src, dst, edge_data = edge
        physical_graph_edges.append((src, dst, {"delay": edge_data["delay"], "forwarding_cost": edge_data["forwarding_cost"], "bandwidth": edge_data["bandwidth"]}))
    return physical_graph_edges

def generate_service_requests(requests_data):
    service_requests = {}
    for req, data in requests_data.items():
        service_requests[req] = {"src": data["src"], "vnfs": data["vnfs"], "dst": data["dst"], "bandwidth": data["bandwidth"], "delay_constraint": data["delay_constraint"]}
    return service_requests

# Read input data from file
nodes_data, edges_data, requests_data = read_input_data('input.txt')

# Generate the dictionaries and lists
physical_graph = generate_physical_graph(nodes_data)
physical_graph_edges = generate_physical_graph_edges(edges_data)
service_requests = generate_service_requests(requests_data)

# AC-3 Algorithm
def revise(physical_graph, service_requests, placement, flow):
    revised = False

    # Check capacity constraints
    for node in physical_graph:
        if sum(placement[node, sfc, vnf] for sfc in service_requests for vnf in service_requests[sfc]["vnfs"]) > physical_graph[node]["capacity"]:
            for sfc in service_requests:
                for vnf in service_requests[sfc]["vnfs"]:
                    if placement[node, sfc, vnf] == 1:
                        placement[node, sfc, vnf] = 0
                        revised = True

    # Check bandwidth constraints
    for src, dst, edge in physical_graph_edges:
        used_bandwidth = sum(flow[src, dst, sfc] for sfc in service_requests)
        if used_bandwidth > edge["bandwidth"]:
            for sfc in service_requests:
                if flow[src, dst, sfc] > 0:
                    flow[src, dst, sfc] = 0
                    revised = True

    # Check delay constraints
    for sfc in service_requests:
        total_delay = sum(flow[src, dst, sfc] * edge["delay"] for src, dst, edge in physical_graph_edges)
        if total_delay > service_requests[sfc]["delay_constraint"]:
            for src, dst, _ in physical_graph_edges:
                if flow[src, dst, sfc] > 0:
                    flow[src, dst, sfc] = 0
                    revised = True

    # Check flow conservation constraints
    for sfc in service_requests:
        for node in physical_graph:
            incoming_flow = sum(flow[src, node, sfc] for src in physical_graph if (src, node) in [(src, dst) for src, dst, _ in physical_graph_edges])
            outgoing_flow = sum(flow[node, dst, sfc] for dst in physical_graph if (node, dst) in [(src, dst) for src, dst, _ in physical_graph_edges])
            if incoming_flow != outgoing_flow:
                for src in physical_graph:
                    if (src, node) in [(src, dst) for src, dst, _ in physical_graph_edges]:
                        flow[src, node, sfc] = 0
                for dst in physical_graph:
                    if (node, dst) in [(src, dst) for src, dst, _ in physical_graph_edges]:
                        flow[node, dst, sfc] = 0
                revised = True

    return revised

def ac3(physical_graph, service_requests):
    # Initialize placement and flow variables
    placement = {(node, sfc, vnf): 0 for node in physical_graph for sfc in service_requests for vnf in service_requests[sfc]["vnfs"]}
    flow = {(src, dst, sfc): 0 for src, dst in [(src, dst) for src, dst, _ in physical_graph_edges] for sfc in service_requests}

    queue = [(node, sfc, vnf) for node in physical_graph for sfc in service_requests for vnf in service_requests[sfc]["vnfs"]]
    queue.extend([(src, dst, sfc) for src, dst in [(src, dst) for src, dst, _ in physical_graph_edges] for sfc in service_requests])

    while queue:
        variable = queue.pop(0)
        if revise(physical_graph, service_requests, placement, flow):
            if isinstance(variable, tuple) and len(variable) == 3:
                queue.extend([(node, sfc, vnf) for node in physical_graph for sfc in service_requests for vnf in service_requests[sfc]["vnfs"] if (node, sfc, vnf) != variable])
            else:
                queue.extend([(src, dst, sfc) for src, dst in [(src, dst) for src, dst, _ in physical_graph_edges] for sfc in service_requests if (src, dst, sfc) != variable])

    return placement, flow

def calculate_cost(physical_graph, service_requests, placement, flow):
    activation_cost = sum(physical_graph[node]["activation_cost"] * any(placement[node, sfc, vnf] for sfc in service_requests for vnf in service_requests[sfc]["vnfs"]) for node in physical_graph)
    deployment_cost = sum(sum(placement[node, sfc, vnf] for node in physical_graph) for sfc in service_requests for vnf in service_requests[sfc]["vnfs"])
    forwarding_cost = sum(edge["forwarding_cost"] * flow[src, dst, sfc] for src, dst, edge in physical_graph_edges for sfc in service_requests)
    sla_violation_cost = sum(service_requests[sfc]["bandwidth"] * (1 - sum(flow[src, dst, sfc] for src, dst in [(src, dst) for src, dst, _ in physical_graph_edges])) for sfc in service_requests)

    total_cost = activation_cost + deployment_cost + forwarding_cost + sla_violation_cost
    return total_cost

def main():
    placement, flow = ac3(physical_graph, service_requests)
    print("Placement:", placement)
    print("Flow:", flow)

    cost = calculate_cost(physical_graph, service_requests, placement, flow)
    print("Overall Cost:", cost)

if __name__ == "__main__":
    main()


end = time.time()
print("Execution time for AC3 :  ",end - start)

Placement: {('R1', 'SFC1', 'VNF1'): 0, ('R1', 'SFC1', 'VNF2'): 0, ('R1', 'SFC1', 'VNF3'): 0, ('R1', 'SFC2', 'VNF4'): 0, ('R1', 'SFC2', 'VNF5'): 0, ('R1', 'SFC3', 'VNF6'): 0, ('R2', 'SFC1', 'VNF1'): 0, ('R2', 'SFC1', 'VNF2'): 0, ('R2', 'SFC1', 'VNF3'): 0, ('R2', 'SFC2', 'VNF4'): 0, ('R2', 'SFC2', 'VNF5'): 0, ('R2', 'SFC3', 'VNF6'): 0, ('R3', 'SFC1', 'VNF1'): 0, ('R3', 'SFC1', 'VNF2'): 0, ('R3', 'SFC1', 'VNF3'): 0, ('R3', 'SFC2', 'VNF4'): 0, ('R3', 'SFC2', 'VNF5'): 0, ('R3', 'SFC3', 'VNF6'): 0, ('R4', 'SFC1', 'VNF1'): 0, ('R4', 'SFC1', 'VNF2'): 0, ('R4', 'SFC1', 'VNF3'): 0, ('R4', 'SFC2', 'VNF4'): 0, ('R4', 'SFC2', 'VNF5'): 0, ('R4', 'SFC3', 'VNF6'): 0, ('R5', 'SFC1', 'VNF1'): 0, ('R5', 'SFC1', 'VNF2'): 0, ('R5', 'SFC1', 'VNF3'): 0, ('R5', 'SFC2', 'VNF4'): 0, ('R5', 'SFC2', 'VNF5'): 0, ('R5', 'SFC3', 'VNF6'): 0, ('R6', 'SFC1', 'VNF1'): 0, ('R6', 'SFC1', 'VNF2'): 0, ('R6', 'SFC1', 'VNF3'): 0, ('R6', 'SFC2', 'VNF4'): 0, ('R6', 'SFC2', 'VNF5'): 0, ('R6', 'SFC3', 'VNF6'): 0}
Flow: {('R1', 'R